<a href="https://colab.research.google.com/github/kimjc95/computational-chemistry/blob/main/Docking_in_Colab(ENG).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Molecular Docking in  Google Colab
2023-12-19 by Joo-Chan Kim at [MSBL](https://msbl.kaist.ac.kr), KAIST

This is a Google Colaboratory Notebook for single ligand docking to a protein receptor. (BSD-3 license)

Please cite DOI:[10.5281/zenodo.13283986](https://zenodo.org/doi/10.5281/zenodo.13283986) if you have used my code in your research.

Please report to [my GitHub](https://github.com/kimjc95/computational-chemistry/issues) or email(kimjoochan@kaist.ac.kr) if you encounter a bug.

-------------------------------

**Things to Prepare :**

1. Ligand's structure file or SMILES string

2. Receptor's structure PDB file or 4-digit PDB ID

3. Google account, Internet connection

-------------------------------

**Warning :**

This notebook is for the docking between a receptor made of (protein with canonical amino acids + metal ions) and a small molecule ligand. For protein-protein docking or protein-nucleic acid docking, please use other tools like AlphaFold3.

In [ ]:
#@title conda environment preparation
#@markdown The runtime will be restarted shortly.

!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.11.0-0/Mambaforge-23.11.0-0-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:14
🔁 Restarting kernel...


In [ ]:
#@title Install Dependencies
#@markdown (takes about 1 min 40 sec)

print('Installing dependencies... ', end='')
import subprocess

subprocess.run("git clone --depth=1 https://github.com/QVina/qvina.git", shell=True)
subprocess.run("chmod -R 755 /content/qvina", shell=True)
subprocess.run("mamba install -c conda-forge rdkit pdbfixer openbabel openmm mdanalysis prolif parmed nglview ipywidgets=7", shell=True)
subprocess.run("pip3 install meeko pdb4amber", shell=True)

from rdkit import RDLogger
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors
from rdkit.Chem import rdFMCS
from rdkit.Chem.Lipinski import RotatableBondSmarts
from pdbfixer import PDBFixer
from openmm import *
from openmm.app import *
from openmm.unit import *
import os
import random
import locale
import warnings
import ipywidgets as widgets
from tqdm.notebook import tqdm_notebook as tqdm
from datetime import datetime
import zipfile
import nglview as nv
import MDAnalysis as mda
import prolif as plf
from MDAnalysis.analysis import distances
from google.colab import output, files
warnings.filterwarnings("ignore")
output.enable_custom_widget_manager()
print("done!")

Installing dependencies... 

done!


In [ ]:
#@title **1. Prepare Ligand.pdbqt**
#@markdown Enter the ligand's SMILES string.<br/>
#@markdown Leave it empty to upload your ligand file.

SMILES = "CC(C=C(C)C=CC(=O)N[O-])C(=O)C1=CC=C(C=C1)N(C)C" #@param {type:"string"}

#@markdown Set the reference pH to determine the protonation state of your ligand.
pH = 7.4 #@param {type:"slider", min:0.0, max:14.0, step:0.1}
#@markdown Select a forcefield to optimize the structure of your ligand.
FFtype = "UFF" #@param ['UFF', 'GAFF', 'Ghemical', 'MMFF94', 'MMFF94s']

print("Optimizing the ligand structure...", end='')
if SMILES == "":
    ligandfile = files.upload()
    ligandfile_name = next(iter(ligandfile))
    subprocess.run(f'obabel {ligandfile_name} -O ligand.sdf -p{pH} --ff {FFtype} --best --minimize --steps 100000 --sd', shell=True)
else:
    subprocess.run(f'obabel -:"{SMILES}" -O ligand.sdf -p{pH} --gen3d --ff {FFtype} --best --minimize --steps 100000 --sd', shell=True)

#@markdown The final structure will be saved to 'ligand.sdf'.
print("done.")

#@markdown Please check your ligand's structure below. If the widget is empty, try with different file format.

view = nv.NGLWidget()
view.add_structure(nv.FileStructure('ligand.sdf'))
view._set_size('700px','500px')
display(view)


Optimizing the ligand structure...done.


NGLWidget()

In [ ]:
#@markdown The rotable bonds will be shown in red.<br/>
#@markdown Among the rotable bonds, type the indices of the bonds you want to rigidify.

mol = Chem.rdmolfiles.SDMolSupplier("ligand.sdf")[0]
Chem.rdmolops.SanitizeMol(mol)

d = Chem.Draw.rdMolDraw2D.MolDraw2DCairo(700, 500)
d.drawOptions().addBondIndices = True
d.drawOptions().annotationFontScale = 0.85

n_rot_bonds = Chem.rdMolDescriptors.CalcNumRotatableBonds(mol)
n_amide_bonds = Chem.rdMolDescriptors.CalcNumAmideBonds(mol)

select_fix = widgets.Text(value='', placeholder='Enter the bonds you want to fix. ex) 11,13',
                          description='Index of the bonds to fix:', style={'description_width': 'initial'},
                          layout=widgets.Layout(width='700px'))

display_rotable = widgets.Text(value=str(n_rot_bonds), description='Number of rotable bonds :',
                               style={'description_width': 'initial'}, layout=widgets.Layout(width='700px'))
display_amide = widgets.Text(value=str(n_amide_bonds), description='Number of amide bonds :',
                             style={'description_width': 'initial'}, layout=widgets.Layout(width='700px'))

rot_atom_pairs = mol.GetSubstructMatches(RotatableBondSmarts)
rot_bonds = list(set([mol.GetBondBetweenAtoms(*ap).GetIdx() for ap in rot_atom_pairs]))

Chem.rdDepictor.Compute2DCoords(mol)
Chem.Draw.rdMolDraw2D.PrepareAndDrawMolecule(d, mol, legend='Rotable in red', highlightBonds=rot_bonds)

d.WriteDrawingText("ligand_2D.png")
with open("ligand_2D.png", "rb") as f:
    data = f.read()
display_image = widgets.Image(value=data, format='png', width=700, height=500)

def recolor_bonds(fix_index):
    bond_colors = {}
    for bond in Chem.rdchem.Mol.GetBonds(mol):
        if bond.GetIdx() in fix_index:
            bond_colors[bond.GetIdx()] = (0.0,0.0,0.8)
        elif bond in rot_bonds:
            bond_colors[bond.GetIdx()] = (0.8,0.0,0.0)
    return bond_colors

def update_rotatable_bond(change):
    new_input = change.new
    if new_input.endswith(','):
        return
    if new_input == '':
        fix_bonds_idx = []
    else:
        fix_bonds_idx = [int(s) for s in new_input.split(',')]
    bond_colors = recolor_bonds(fix_bonds_idx)
    d.ClearDrawing()
    Chem.Draw.rdMolDraw2D.PrepareAndDrawMolecule(d, mol, legend='Rotable in red, Fixed in blue',
                                                 highlightBonds=rot_bonds, highlightBondColors=bond_colors)
    rot_bond_count = 0
    for bond in rot_bonds:
        if bond not in fix_bonds_idx:
            rot_bond_count += 1
    display_rotable.value = str(rot_bond_count)
    d.WriteDrawingText("ligand_2D.png")
    with open("ligand_2D.png", "rb") as f:
        data = f.read()
    display_image.value = data

select_fix.observe(update_rotatable_bond, names='value')
display(select_fix, display_rotable, display_amide, display_image)

Text(value='', description='Index of the bonds to fix:', layout=Layout(width='700px'), placeholder='Enter the …

Text(value='6', description='Number of rotable bonds :', layout=Layout(width='700px'), style=DescriptionStyle(…

Text(value='1', description='Number of amide bonds :', layout=Layout(width='700px'), style=DescriptionStyle(de…

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02\xbc\x00\x00\x01\xf4\x08\x02\x00\x00\x00P;i\x88\x…

In [ ]:
# @markdown Preparing ligand.pdbqt by prepare_ligand.py.

# 원래 SMARTS로 입력해야 하지만 귀찮으므로 SMILES로 fixed bond 지정
SMILES = Chem.rdmolfiles.MolToSmiles(mol)
rotable = ""

if select_fix.value == '':
    fix_bonds_idx = []
else:
    fix_bonds_idx = [int(s) for s in select_fix.value.split(',')]

for i in fix_bonds_idx:
    bond = mol.GetBondWithIdx(i)
    a1 = bond.GetBeginAtom().GetIdx()+1 # RDKit은 0부터 셈
    a2 = bond.GetEndAtom().GetIdx()+1
    rotable = rotable + ' -r "'+SMILES+'" -b '+str(a1)+" "+str(a2)

subprocess.run(f'mk_prepare_ligand.py -i ligand.sdf -o ligand.pdbqt{rotable}', shell=True)

if os.path.exists('ligand.pdbqt'):
    print('ligand PDBQT file has been generated.')
else:
    print('Failed to generate ligand PDBQT file.')

ligand PDBQT file has been generated.


In [ ]:
# @title **2. Prepare Receptor.pdbqt**
# @markdown Enter the PDB ID of the receptor. You can leave it empty to upload your PDB file.<br/>
PDB_ID = "1c3s" # @param {type: "string"}
#@markdown Get used to the NGLViewer below!<br/><br/>
#@markdown Left drag - rotate<br/>Right drag - translate<br/>Wheel scroll - zoom in/out <br/>Left long click - recenter

PDB_ID = PDB_ID.upper()

view2 = nv.NGLWidget()
view2._set_size('1000px','750px')

if PDB_ID == "":
    receptorfile = files.upload()
    receptorfile_name = next(iter(receptorfile))
    fixer = PDBFixer(filename=next(iter(receptorfile)))
else:
    fixer = PDBFixer(pdbid=PDB_ID)

with open('receptor-unprocessed.pdb', 'w') as f:
    PDBFile.writeFile(fixer.topology, fixer.positions, f)

view2.add_structure(nv.FileStructure('receptor-unprocessed.pdb'))
view2.add_spacefill("ion")
display(view2)

NGLWidget()

In [ ]:
amino_acids = ['ALA', 'CYS', 'ASP', 'GLU', 'PHE', 'GLY', 'HIS', 'ILE', 'LYS', 'LEU',
               'MET', 'ASN', 'PRO', 'GLN', 'ARG', 'SER', 'THR', 'VAL', 'TRP', 'TYR']
nucleic_acids = ['A', 'T', 'G', 'C', 'U', 'I', 'DA', 'DT', 'DG', 'DC', 'DI']

#@markdown In the above widget, chains are shown in different colors.
#@markdown Enter the chains you want to preserve.
chains_to_keep = 'A' # @param {type:"string"}
#@markdown To maintain metal ions in your system, enter the metal's residue name and its charge separated by colon (:).
#@markdown For multiple metal ions, separate them with commas (,).
metal_to_keep = 'ZN:2' # @param {type:"string"}
# @markdown The fixed receptor structure will be saved to 'receptor-fixed.pdb'.

if metal_to_keep != '':
    metal_residues = metal_to_keep.split(',')
    metals_to_keep = {}
    for metal in metal_residues:
        metal_info = metal.split(':')
        metals_to_keep[metal_info[0]] = metal_info[1]

print("Fixing the pdb file...", end='')
chains_to_delete = []

for chain in fixer.topology.chains():
    if chain.id not in chains_to_keep:
        chains_to_delete.append(chain.index)

fixer.removeChains(chains_to_delete)
fixer.findNonstandardResidues()
fixer.replaceNonstandardResidues()
fixer.findMissingResidues()
terminal_missing_res = []
chains = list(fixer.topology.chains())
for key in fixer.missingResidues.keys():
    if key[1] == 0 or key[1] == len(list(chains[key[0]].residues())):
        terminal_missing_res.append(key)
for key in terminal_missing_res:
    del fixer.missingResidues[key]
fixer.findMissingAtoms()
fixer.addMissingAtoms()
fixer.addMissingHydrogens(pH)

model = Modeller(fixer.topology, fixer.positions)
model.deleteWater()

hetero_res = []
for res in model.topology.residues():
    if res.name not in (amino_acids+nucleic_acids+list(metals_to_keep.keys())):
        hetero_res.append(res)
model.delete(hetero_res)

n_metals = 0
for res in model.topology.residues():
    if res.name in metals_to_keep:
        n_metals += 1

print("done.")

with open('receptor-fixed.pdb', 'w') as f:
    PDBFile.writeFile(model.topology, model.positions, f)

u = mda.Universe('receptor-fixed.pdb')
com = u.select_atoms('all').center_of_geometry()
u.atoms.translate(-com)

for chain in u.segments:
    first_residue = chain.residues[0]
    n_terminal_h = first_residue.atoms.select_atoms('name H')
    if len(n_terminal_h) > 0:
        n_terminal_h[0].name = 'H1'

if n_metals > 0:
    u.add_TopologyAttr('charge')
    for res in u.residues:
        if res.resname in metals_to_keep:
            res.atoms[0].charge =  float(metals_to_keep[res.resname])
    u.select_atoms(f'resname {" ".join(list(metals_to_keep.keys()))}').write('metals.pdbqt')

u.select_atoms('protein or nucleic').write('receptor-centered.pdb')

view3 = nv.NGLWidget()
view3._set_size('1000px','750px')
view3.add_structure(nv.FileStructure('receptor-centered.pdb'))
view3.add_licorice()
if n_metals > 0:
    view3.add_structure(nv.FileStructure('metals.pdbqt'))
    view3.add_spacefill("ion")
view3.add_surface(lowResolution= True, smooth=1,opacity=0.5)
display(view3)

Fixing the pdb file...done.


NGLWidget()

In [ ]:
# @markdown In the above widget, select an atom to set it as a center for the grid box.<br/>
# @markdown When an atom is selected, its info will be shown in white at the upper left corner.<br/>
# @markdown Or you can enter the coordinates of the box center manually below.

center_x = "" #@param {type:"string"}
center_y = "" #@param {type:"string"}
center_z = "" #@param {type:"string"}

# @markdown If you are not aware of the putative binding site, check the box below and leave the above boxes empty.

blind_docking = False #@param {type:"boolean"}

if center_x != "" and center_y != "" and center_z != "":
    try:
        center_x = float(center_x)
        center_y = float(center_y)
        center_z = float(center_z)
    except:
        print("Box center coordinates are not read.")

elif blind_docking:
    center_x = 0.0
    center_y = 0.0
    center_z = 0.0

elif view3.picked != {}:
    center_x = view3.picked["atom1"]["x"]
    center_y = view3.picked["atom1"]["y"]
    center_z = view3.picked["atom1"]["z"]

else:
    print("The atom for the box center has not been selected.")

print(f"The coordinate for the box center is x:{center_x:.1f} A, y:{center_y:.1f} A, z:{center_z:.1f} A.")


The coordinate for the box center is x:-6.1 A, y:0.3 A, z:-3.7 A.


In [ ]:
# @markdown Adjust the box position/dimensions. <br/>
# @markdown The x-axis is shown in red, y-axis in green, and z-axis in blue. <br/>
# @markdown The default box size is calculated from the ligand size.

def find_ligand_dim():
    ligand = Chem.rdmolfiles.SDMolSupplier("ligand.sdf")[0]
    xs = []
    ys = []
    zs = []

    for coord in ligand.GetConformer().GetPositions():
        xs.append(coord[0])
        ys.append(coord[1])
        zs.append(coord[2])

    dx = max(xs)-min(xs)
    dy = max(ys)-min(ys)
    dz = max(zs)-min(zs)

    return int(sqrt(dx*dx+dy*dx+dz*dz))


view4 = nv.NGLWidget()
view4._set_size('1000px','750px')
view4.add_structure(nv.FileStructure('receptor-centered.pdb'))
if n_metals > 0:
    view4.add_structure(nv.FileStructure('metals.pdbqt'))
    view4.add_spacefill("ion")
view4.add_licorice()
view4.add_surface(lowResolution= True, smooth=1,opacity=0.4)

x_select = widgets.BoundedFloatText(value=center_x, min=-1000, max=1000, step=0.5, description="center_x (A): ",
                                    continuous_update=False, style={'description_width': 'initial'}, layout=widgets.Layout(width='1000px'))

y_select = widgets.BoundedFloatText(value=center_y, min=-1000, max=1000, step=0.5, description="center_y (A): ",
                                    continuous_update=False, style={'description_width': 'initial'}, layout=widgets.Layout(width='1000px'))

z_select = widgets.BoundedFloatText(value=center_z, min=-1000, max=1000, step=0.5, description="center_z (A): ",
                                    continuous_update=False, style={'description_width': 'initial'}, layout=widgets.Layout(width='1000px'))

r = find_ligand_dim()

l_select = widgets.BoundedFloatText(value=r*2, min=r, max=1000, step=0.5, description="length_x (A) : ",
                                    continuous_update=False, style={'description_width': 'initial'}, layout=widgets.Layout(width='1000px'))

w_select = widgets.BoundedFloatText(value=r*2, min=r, max=1000, step=0.5, description="length_y (A) : ",
                                    continuous_update=False, style={'description_width': 'initial'}, layout=widgets.Layout(width='1000px'))

h_select = widgets.BoundedFloatText(value=r*2, min=r, max=1000, step=0.5, description="length_z (A) : ",
                                    continuous_update=False, style={'description_width': 'initial'}, layout=widgets.Layout(width='1000px'))


def draw_box(v, x, y, z, l, w, h, runtime):
    if runtime > 0:
        v._execute_js_code(f"""
        this.stage.removeComponent(this.stage.getComponentsByName("vertice1").first);
        this.stage.removeComponent(this.stage.getComponentsByName("vertice2").first);
        this.stage.removeComponent(this.stage.getComponentsByName("vertice3").first);
        this.stage.removeComponent(this.stage.getComponentsByName("vertice4").first);
        this.stage.removeComponent(this.stage.getComponentsByName("vertice5").first);
        this.stage.removeComponent(this.stage.getComponentsByName("vertice6").first);
        this.stage.removeComponent(this.stage.getComponentsByName("vertice7").first);
        this.stage.removeComponent(this.stage.getComponentsByName("vertice8").first);
        this.stage.removeComponent(this.stage.getComponentsByName("vertice9").first);
        this.stage.removeComponent(this.stage.getComponentsByName("vertice10").first);
        this.stage.removeComponent(this.stage.getComponentsByName("vertice11").first);
        this.stage.removeComponent(this.stage.getComponentsByName("vertice12").first)
        """)
    corner1 = [x+l/2, y+w/2, z+h/2]
    corner2 = [x-l/2, y+w/2, z+h/2]
    corner3 = [x-l/2, y-w/2, z+h/2]
    corner4 = [x+l/2, y-w/2, z+h/2]
    corner5 = [x+l/2, y+w/2, z-h/2]
    corner6 = [x-l/2, y+w/2, z-h/2]
    corner7 = [x-l/2, y-w/2, z-h/2]
    corner8 = [x+l/2, y-w/2, z-h/2]
    v._execute_js_code(
        f"""
        this.addShape("vertice1", [["cylinder", {corner1}, {corner2}, [1,1,1], [0.1]]]);
        this.addShape("vertice2", [["cylinder", {corner2}, {corner3}, [1,1,1], [0.1]]]);
        this.addShape("vertice3", [["cylinder", {corner3}, {corner4}, [1,1,1], [0.1]]]);
        this.addShape("vertice4", [["cylinder", {corner4}, {corner1}, [1,1,1], [0.1]]]);
        this.addShape("vertice5", [["cylinder", {corner1}, {corner5}, [1,1,1], [0.1]]]);
        this.addShape("vertice6", [["cylinder", {corner2}, {corner6}, [1,1,1], [0.1]]]);
        this.addShape("vertice7", [["cylinder", {corner3}, {corner7}, [0,0,1], [0.2]]]);
        this.addShape("vertice8", [["cylinder", {corner4}, {corner8}, [1,1,1], [0.1]]]);
        this.addShape("vertice9", [["cylinder", {corner5}, {corner6}, [1,1,1], [0.1]]]);
        this.addShape("vertice10", [["cylinder", {corner6}, {corner7}, [0,1,0], [0.2]]]);
        this.addShape("vertice11", [["cylinder", {corner7}, {corner8}, [1,0,0], [0.2]]]);
        this.addShape("vertice12", [["cylinder", {corner8}, {corner5}, [1,1,1], [0.1]]])
        """)

def update_x(change):
    x = change.new
    y = y_select.value
    z = z_select.value
    l = l_select.value
    w = w_select.value
    h = h_select.value
    draw_box(view4, x, y, z, l, w, h, 1)

def update_y(change):
    x = x_select.value
    y = change.new
    z = z_select.value
    l = l_select.value
    w = w_select.value
    h = h_select.value
    draw_box(view4, x, y, z, l, w, h, 1)

def update_z(change):
    x = x_select.value
    y = y_select.value
    z = change.new
    l = l_select.value
    w = w_select.value
    h = h_select.value
    draw_box(view4, x, y, z, l, w, h, 1)

def update_l(change):
    x = x_select.value
    y = y_select.value
    z = z_select.value
    l = change.new
    w = w_select.value
    h = h_select.value
    draw_box(view4, x, y, z, l, w, h, 1)

def update_w(change):
    x = x_select.value
    y = y_select.value
    z = z_select.value
    l = l_select.value
    w = change.new
    h = h_select.value
    draw_box(view4, x, y, z, l, w, h, 1)

def update_h(change):
    x = x_select.value
    y = y_select.value
    z = z_select.value
    l = l_select.value
    w = w_select.value
    h = change.new
    draw_box(view4, x, y, z, l, w, h, 1)

draw_box(view4, x_select.value, y_select.value, z_select.value, l_select.value, w_select.value, h_select.value, 0)
x_select.observe(update_x, names='value')
y_select.observe(update_y, names='value')
z_select.observe(update_z, names='value')
l_select.observe(update_l, names='value')
w_select.observe(update_w, names='value')
h_select.observe(update_h, names='value')

center_x = x_select.value
center_y = y_select.value
center_z = z_select.value
len_x = l_select.value
len_y = w_select.value
len_z = h_select.value

display(x_select, y_select, z_select, l_select, w_select, h_select, view4)

BoundedFloatText(value=-6.076000213623047, description='center_x (A): ', layout=Layout(width='1000px'), max=10…

BoundedFloatText(value=0.27000001072883606, description='center_y (A): ', layout=Layout(width='1000px'), max=1…

BoundedFloatText(value=-3.6600000858306885, description='center_z (A): ', layout=Layout(width='1000px'), max=1…

BoundedFloatText(value=30.0, description='length_x (A) : ', layout=Layout(width='1000px'), max=1000.0, min=15.…

BoundedFloatText(value=30.0, description='length_y (A) : ', layout=Layout(width='1000px'), max=1000.0, min=15.…

BoundedFloatText(value=30.0, description='length_z (A) : ', layout=Layout(width='1000px'), max=1000.0, min=15.…

NGLWidget()

In [ ]:
# @markdown Enter the flexible residues as
# @markdown "Chain ID:Residue number"<br/>
# @markdown For more than one flexible residues, separate them with commas without blanks. <br/>
# @markdown If there is none, leave it empty.

flexible_residues = "A:199" # @param {type:"string"}

flexFlag = False
flex_res = ""
selection = ""

if flexible_residues != "":
    try:
        for i, fr in enumerate(flexible_residues.split(',')):
            resinfo = fr.split(':')
            for res in u.residues:
                if res.segid == resinfo[0] and res.resnum == int(resinfo[1]):
                    flex_res = flex_res + f' -f {resinfo[0]}:{res.resname}:{resinfo[1]}'
                    selection = selection + f'(:{resinfo[0]} and {resinfo[1]})'
                    if i != (len(fr)-1):
                        selection = selection + " or "
            flexFlag = True
    except:
        print("Failed to recognize the flexible residue info.")

view5 = nv.NGLWidget()
view5._set_size('1000px','750px')
view5.add_structure(nv.FileStructure('receptor-centered.pdb'))
if n_metals > 0:
    view5.add_structure(nv.FileStructure('metals.pdbqt'))
    view5.add_spacefill("ion")
view5.add_surface(lowResolution= True, smooth=1,opacity=0.4)

center_x = x_select.value
center_y = y_select.value
center_z = z_select.value
len_x = l_select.value
len_y = w_select.value
len_z = h_select.value

draw_box(view5, center_x, center_y, center_z, len_x, len_y, len_z, 0)

if flexFlag:
    view5.add_licorice(selection=selection, color='red')
    view5.add_surface(selection=selection, color='red',
                      lowResolution=True, smooth=1,opacity=0.8)
    display(view5)

else:
    print("No flexible residues set.")

NGLWidget()

In [ ]:
# @markdown Prepare receptor.pdbqt.

subprocess.run('obabel receptor-centered.pdb -O receptor-charged.pdbqt -xr --partialcharge gasteiger', shell=True)
subprocess.run(f'mk_prepare_receptor.py --pdbqt receptor-charged.pdbqt -o receptor.pdbqt --box_size {len_x} {len_y} {len_z} --box_center {center_x} {center_y} {center_z}{flex_res} --skip_gpf', shell=True)

if flexFlag:
    target = 'receptor_rigid.pdbqt'
else:
    target = 'receptor.pdbqt'

if os.path.exists(f'{target}'):
    print('receptor PDBQT file generated.')
else:
    print('Failed to generate receptor PDBQT file.')

if n_metals > 0:
    subprocess.run(f'tail -n {n_metals} metals.pdbqt >> {target}', shell=True)

receptor PDBQT file generated.


In [ ]:
# @title **3. Dock with QuickVina**
output = "ligand_out.pdbqt"
# @markdown Set the exhaustiveness value. For blind dockings, increase it to larger values.
exhaustiveness = 16 #@param {type:"slider", min:8, max:512, step:8}
# @markdown The docking log below will also be saved at docking_log.txt.

if flexFlag:
    rigid = "receptor = receptor_rigid.pdbqt\n"
    flex_rec = "flex = receptor_flex.pdbqt\n"
else:
    rigid = "receptor = receptor.pdbqt\n"
    flex_rec = ""

with open('config', 'w') as f:
    f.write(rigid)
    f.write("ligand = ligand.pdbqt\n")
    f.write("center_x = "+str(center_x)+'\n')
    f.write("center_y = "+str(center_x)+'\n')
    f.write("center_z = "+str(center_x)+'\n')
    f.write("size_x = "+str(len_x)+'\n')
    f.write("size_y = "+str(len_y)+'\n')
    f.write("size_z = "+str(len_z)+'\n')
    f.write("num_modes = 10\n")
    f.write("out = "+output+'\n')
    f.write("log = docking_log.txt\n")
    f.write("exhaustiveness = "+str(exhaustiveness)+'\n')
    f.write(flex_rec)

if blind_docking:
    p = subprocess.Popen('/content/qvina/bin/qvina-w --config config', shell=True, bufsize=0, stdout=subprocess.PIPE, encoding='utf-8')
else:
    p = subprocess.Popen('/content/qvina/bin/qvina2.1 --config config', shell=True, bufsize=0, stdout=subprocess.PIPE, encoding='utf-8')

while p.poll() == None:
    out = p.stdout.read(1)
    print(out, end='')

############################################################################
# If you used Quick Vina 2 in your work, please cite:                      #
#                                                                          #
# Amr Alhossary, Stephanus Daniel Handoko, Yuguang Mu, and Chee-Keong Kwoh,#
# Fast, Accurate, and Reliable Molecular Docking with QuickVina 2,         #
# Bioinformatics (2015), doi: 10.1093/bioinformatics/btv082                #
#                                                                          #
# You are also encouraged to cite Quick Vina 1:                            #
# Stephanus Daniel Handoko, Xuchang Ouyang, Chinh Tran To Su, Chee Keong   #
# Kwoh, Yew Soon Ong,                                                      #
# QuickVina: Accelerating AutoDock Vina Using Gradient-Based Heuristics for#
# Global Optimization,                                                     #
# IEEE/ACM Transactions on Computational Biology and Bioinformatics,vol.9, #

In [ ]:
# @title **4. Results analysis**
#@markdown If the number of modes in the above docking log is less than 10, please adjust the num_models below accordingly. <br/>
#@markdown This cell may take a while if your system is large.

num_models = "10" #@param [1,2,3,4,5,6,7,8,9,10]

def typeConverter(universe):
    type2element = {'H':'H','C':'C','A':'C','N':'N','P':'P','S':'S','Br':'Br','BR':'Br','I':'I','F':'F','Cl':'Cl','CL':'Cl',
                    'NA':'N','OA':'O','SA':'S','OS':'O','NS':'N','HD':'H','HS':'H','Mg':'Mg','Ca':'Ca','Fe':'Fe','Zn':'Zn','O':'O',
                    'Mn':'Mn','MG':'Mg','CA':'Ca','FE':'Fe','ZN':'Zn','MN':'Mn','G':'C','J':'C','Q':'C','GA':'C', 'K':'K'}

    universe.add_TopologyAttr('element')
    for atom in universe.atoms:
        element = type2element[atom.type]
        atom.type = element
        atom.element = element

    return universe


def flex_res_merger(universe, flexible_residues):

    flex_res = []
    fr = flexible_residues.split(',')
    for res in fr:
        res_info = res.split(':')
        flex_res.append(u.select_atoms(f'chainID {res_info[0]} and resname {res_info[1]} and resnum {res_info[2]}'))

    for i in range(len(flex_res)):
        res = universe.atoms.split('residue')[-i-1]
        a = res.select_atoms("name CA")
        dists = []
        for res2 in flex_res:
            resids_A, resids_B, dist = distances.dist(a,res2.select_atoms("name C"))
            dists.append(list(dist)[0])
        nearest = flex_res[dists.index(min(dists))][0]
        res.residues.resnames = nearest.resname
        res.residues.resnums = nearest.resnum

    return universe

def set_conformer_and_add_H(mol_wH, mol_A, mol_B):
    mol_C = Chem.Mol(mol_wH)
    conf_A = mol_A.GetConformer()
    conf_B = mol_B.GetConformer()

    map = {}
    for idx_A, coord_A in enumerate(conf_A.GetPositions()):
        dr = []
        for idx_C, coord_C in enumerate(mol_C.GetConformer().GetPositions()):
            dx = (coord_A[0] - coord_C[0])
            dy = (coord_A[1] - coord_C[1])
            dz = (coord_A[2] - coord_C[2])
            dr.append(sqrt(dx*dx+dy*dy+dz*dz))
        map[idx_A] = dr.index(min(dr))

    mol_C.RemoveAllConformers()
    mol_C.AddConformer(Chem.Conformer(mol_C.GetNumAtoms()))

    xs = []
    ys = []
    zs = []
    for idx, coord in enumerate(conf_B.GetPositions()):
        mol_C.GetConformer().SetAtomPosition(map[idx], coord)
        xs.append(coord[0])
        ys.append(coord[1])
        zs.append(coord[2])

    size_x = max(xs) - min(xs)
    size_y = max(ys) - min(ys)
    size_z = max(zs) - min(zs)

    for idx in range(mol_C.GetNumAtoms()):
        if idx not in map.values():
            x = random.random()*size_x + min(xs)
            y = random.random()*size_y + min(ys)
            z = random.random()*size_z + min(zs)
            mol_C.GetConformer().SetAtomPosition(idx, (x,y,z))

    RDLogger.DisableLog('rdApp.*')

    for i in range(1000):
        if Chem.rdForceFieldHelpers.UFFOptimizeMolecule(mol_C, maxIters=3) == 0:
            break
        for key, value in map.items():
            mol_C.GetConformer().SetAtomPosition(value, list(conf_B.GetPositions())[key])

    Chem.rdmolops.AssignStereochemistryFrom3D(mol_C)

    return mol_C

rdkit_mol = Chem.rdmolfiles.SDMolSupplier("ligand.sdf")[0]
rdkit_mol = Chem.AddHs(rdkit_mol)
pdbqt_mol = typeConverter(mda.Universe("ligand.pdbqt")).atoms
pdbqt_mol.guess_bonds(fudge_factor=0.65)
lig_wo_pose = pdbqt_mol.convert_to("RDKIT")

ligs = typeConverter(mda.Universe("ligand_out.pdbqt"))
size = len(ligs.atoms)/int(num_models)

flex_res = []
if flexFlag:
    rec = typeConverter(mda.Universe("receptor_rigid.pdbqt")).atoms.convert_to('PARMED')
else:
    rec = typeConverter(mda.Universe("receptor.pdbqt")).atoms.convert_to('PARMED')

print('Converting the ligand.pdbqt file to complex pdb file...', end='')
poses = []
for i in tqdm(range(int(num_models))):
    b_i = int(i*size)
    e_i = int((i+1)*size-1)

    u_complex = mda.Universe(rec + ligs.select_atoms("index "+str(b_i)+':'+str(e_i), sorted=False).convert_to("PARMED"))

    if flexFlag:
        u_complex = flex_res_merger(u_complex, flexible_residues)

    if n_metals > 0:
        selection_string = f'not protein and not nucleic and not resname {" ".join(list(metals_to_keep.keys()))}'
    else:
        selection_string = 'not protein and not nucleic'
    lig = typeConverter(u_complex).select_atoms(selection_string)
    lig.guess_bonds(fudge_factor=0.65)
    lig_w_pose = lig.convert_to("RDKIT")
    lmol = set_conformer_and_add_H(rdkit_mol, lig_wo_pose, lig_w_pose)

    rec_atoms = typeConverter(u_complex).select_atoms('protein or nucleic')
    rec_atoms.guess_bonds(fudge_factor=0.6)
    rec_atoms.write("receptor.pdb")

    fixer = PDBFixer("receptor.pdb")
    fixer.addMissingHydrogens(pH)

    if not flexFlag:
        if i == 0:
            PDBFile.writeFile(fixer.topology, fixer.positions, "receptor.pdb")
        rmol = mda.Universe("receptor.pdb")
    else:
        PDBFile.writeFile(fixer.topology, fixer.positions, f'receptor_pose{i+1}.pdb')
        rmol = mda.Universe(f'receptor_pose{i+1}.pdb')

    c = rmol.atoms.convert_to("PARMED") + mda.Universe(lmol).atoms.convert_to("PARMED")

    if n_metals > 0:
        c += mda.Universe('metals.pdbqt').atoms.convert_to("PARMED")

    poses.append(mda.Universe(c))
    PDBFile.writeFile(c.topology, c.positions, f'complex_pose{i+1}.pdb')

print("done.")

print("Downloading result files.")
filename = f'docking_result_{datetime.today().strftime("%Y-%m-%d")}.zip'

with zipfile.ZipFile(filename, 'w') as zip_file:
    for i in range(int(num_models)):
        zip_file.write(f'complex_pose{i+1}.pdb')
    zip_file.write('docking_log.txt')
    zip_file.write('ligand.pdbqt')
    if flexFlag:
        zip_file.write('receptor_rigid.pdbqt')
        zip_file.write('receptor_flex.pdbqt')
    else:
        zip_file.write('receptor.pdbqt')
    zip_file.write('receptor-fixed.pdb')
    zip_file.write('ligand.sdf')

files.download(filename)

Converting the ligand.pdbqt file to complex pdb file...

  0%|          | 0/10 [00:00<?, ?it/s]

done.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# @markdown Select a pose to view.

pose = "1" # @param [1,2,3,4,5,6,7,8,9,10]
view_surface = True #@ param {type:"boolean"}

view5 = nv.NGLWidget()
view5._set_size('1000px', '750px')
view5.add_structure(nv.MDAnalysisTrajectory(poses[int(pose)-1]))
if n_metals > 0:
    view5.add_spacefill(selection="ion")
if view_surface:
    view5.add_surface(selection="protein or nucleic or ion", smooth=1, opacity=0.5)
view5

NGLWidget()

In [ ]:
# @markdown You can visualize the receptor-ligand interaction in 2D below.

if n_metals > 0:
    selection_string = f'protein or nucleic or (resname {" ".join(list(metals_to_keep.keys()))})'
else:
    selection_string = 'protein or nucleic'

prot = poses[int(pose)-1].select_atoms(selection_string)

if n_metals > 0:
    selection_string = f'not protein and not nucleic and not (resname {" ".join(list(metals_to_keep.keys()))})'
else:
    selection_string = 'not protein and not nucleic'

lig = poses[int(pose)-1].select_atoms(selection_string)

fp = plf.Fingerprint()
fp.run(poses[int(pose)-1].trajectory, lig, prot)
fp.plot_lignetwork(plf.Molecule.from_rdkit(lig.convert_to("RDKIT")))

  0%|          | 0/1 [00:00<?, ?it/s]

Tools I used

*  RDKit
> RDKit: Open-source cheminformatics. https://www.rdkit.org

* OpenBabel
> N. M. O'Boyle, M. Banck, C. A. James, C. Morley, T. Vandermeersch, and G. R. Hutchison (2011) "Open Babel: An open chemical toolbox" J. Cheminf. 3:33. DOI:[10.1186/1758-2946-3-33](https://doi.org/10.1186/1758-2946-3-33)

*  OpenMM
> P. Eastman, J. Swails, J. D. Chodera, R. T. McGibbon, Y. Zhao, K. A. Beauchamp, L.-P. Wang, A. C. Simmonett, M. P. Harrigan, C. D. Stern, R. P. Wiewiora, B. R. Brooks, and V. S. Pande. (2017)“OpenMM 7: Rapid development of high performance algorithms for molecular dynamics.” PLOS Comp. Biol. 13(7): e1005659. DOI:[10.1371/journal.pcbi.1005659](https://doi.org/10.1371/journal.pcbi.1005659)

*   AutoDock Suite
> Forli, S., Huey, R., Pique, M. E., Sanner, M. F., Goodsell, D. S., & Olson, A. J. (2016) "Computational protein–ligand docking and virtual drug screening with the AutoDock suite" Nature protocols, 11(5), 905-919. DOI:[10.1038/nprot.2016.051](https://doi.org/10.1038/nprot.2016.051)

*   QuickVina2
> Amr Alhossary, Stephanus Daniel Handoko, Yuguang Mu, and Chee-Keong Kwoh. (2015) "Fast, Accurate, and Reliable Molecular Docking with QuickVina 2" Bioinformatics 31(13): 2214-2216. DOI:[10.1093/bioinformatics/btv082](https://doi.org/10.1093/bioinformatics/btv082)

* QuickVina-W
> N. M. Hassan, A. A. Alhossary, Y. Mu, and C.-K. Kwoh. (2017) "Protein-Ligand Blind Docking Using QuickVina-W with Inter-Process Spatio-Temporal Integration" Nat. Sci. Rep. 7(1): 15451. DOI:[10.1038/s41598-017-15571-7](https://doi.org/10.1038/s41598-017-15571-7)

*  NGLViewer
> Hai Nguyen, David A Case, Alexander S Rose. (2018) "NGLview - Interactive molecular graphics for Jupyter notebooks" Bioinformatics 34(7): 1241-1242. DOI:[10.1093/bioinformatics/btx789](https://doi.org/10.1093/bioinformatics/btx789)

* MDAnalysis
> N. Michaud-Agrawal, E. J. Denning, T. B. Woolf, and O. Beckstein. (2011) "MDAnalysis: A Toolkit for the Analysis of Molecular Dynamics Simulations." J. Comput. Chem. 32(10): 2319-2327, DOI:[10.1002/jcc.21787](https://doi.org/10.1002/jcc.21787)

* ParmEd
> M. R. Shirts, C. Klein, J. M. Swails, J. Yin, M. K. Gilson, D. L. Mobley, D. A. Case & E. D. Zhong. (2017) "Lessons learned from comparing molecular dynamics engines on the SAMPL5 dataset" J. Comput. Aided Mol. Des. 31, 147-161. DOI:[10.1007/s10822-016-9977-1](https://doi.org/10.1007/s10822-016-9977-1)

*  ProLIF
> Bouysset, C., Fiorucci, S. (2021) "ProLIF: a library to encode molecular interactions as fingerprints." J. Cheminform. 13: 72. DOI:[10.1186/s13321-021-00548-6](https://doi.org/10.1186/s13321-021-00548-6)